In [1]:

# import tkinter as tk
# from tkinter import ttk
# import csv

# def save_to_csv(event=None):
#     pass

# def search_in_csv():
#     pass

# def delete_from_csv():
#     pass

# def clear_fields():
#     entry_product.delete(0, tk.END)
#     entry_number.delete(0, tk.END)
#     entry_price.delete(0, tk.END)
#     entry_search.delete(0, tk.END)

# root = tk.Tk()
# root.title("상품 관리 시스템")
# root.geometry("600x400")

# style = ttk.Style()
# style.configure("TLabel", font=("Arial", 12))
# style.configure("TButton", font=("Arial", 12), padding=10)
# style.configure("TEntry", font=("Arial", 12))

# frame = ttk.Frame(root, padding="10 10 10 10")
# frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))

# label_product = ttk.Label(frame, text="상품명:")
# label_product.grid(row=1, column=0, padx=10, pady=10, sticky='e')

# entry_product = ttk.Entry(frame, width=30)
# entry_product.grid(row=1, column=1, padx=10, pady=10)

# label_number = ttk.Label(frame, text="번호:")
# label_number.grid(row=2, column=0, padx=10, pady=10, sticky='e')

# entry_number = ttk.Entry(frame, width=30)
# entry_number.grid(row=2, column=1, padx=10, pady=10)

# label_price = ttk.Label(frame, text="가격:")
# label_price.grid(row=3, column=0, padx=10, pady=10, sticky='e')

# entry_price = ttk.Entry(frame, width=30)
# entry_price.grid(row=3, column=1, padx=10, pady=10)

# btn_save = ttk.Button(frame, text="저장", command=save_to_csv)
# btn_save.grid(row=4, column=1, padx=10, pady=10)

# label_search = ttk.Label(frame, text="검색 또는 삭제 (상품, 번호):")
# label_search.grid(row=5, column=0, padx=10, pady=10, sticky='e')

# entry_search = ttk.Entry(frame, width=30)
# entry_search.grid(row=5, column=1, padx=10, pady=10)

# btn_search = ttk.Button(frame, text="검색", command=search_in_csv)
# btn_search.grid(row=6, column=1, padx=10, pady=10)

# btn_delete = ttk.Button(frame, text="삭제", command=delete_from_csv)
# btn_delete.grid(row=7, column=1, padx=10, pady=10)

# btn_clear = ttk.Button(frame, text="내용 삭제", command=clear_fields)
# btn_clear.grid(row=8, column=1, padx=10, pady=10)

# root.bind('<Return>', save_to_csv)

# root.mainloop()

In [1]:
import tkinter as tk
from tkinter import messagebox, filedialog, Toplevel, Text, Scrollbar, VERTICAL, RIGHT, Y, END
import csv
import os

csv_file = None

def set_save_path():
    global csv_file
    file_path = filedialog.asksaveasfilename(defaultextension=".csv", filetypes=[("CSV files", "*.csv")])
    if file_path:
        csv_file = file_path
        messagebox.showinfo("파일 경로 설정", f"CSV 파일 경로가 설정되었습니다:\n{csv_file}")

def save_to_csv(event=None):  
    global csv_file
    if csv_file is None:
        messagebox.showwarning("경로 오류", "먼저 파일 저장 경로를 설정하세요.")
        return
    
    product_name = entry_product.get()
    product_number = entry_number.get()
    product_price = entry_price.get()  
    
    if product_name and product_number and product_price:
        with open(csv_file, 'a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([product_name, product_number, product_price])  
        messagebox.showinfo("저장 완료", f"{product_name} - {product_number} - {product_price} 저장됨.")
        entry_product.delete(0, tk.END)
        entry_number.delete(0, tk.END)
        entry_price.delete(0, tk.END)  
    else:
        messagebox.showwarning("입력 오류", "모든 필드를 입력하세요.")

def search_in_csv():
    global csv_file
    if csv_file is None:
        messagebox.showwarning("경로 오류", "먼저 파일 저장 경로를 설정하세요.")
        return

    search_term = entry_search.get().lower()  
    results = []
    
    if os.path.exists(csv_file):
        with open(csv_file, 'r') as file:
            reader = csv.reader(file)
            for row in reader:
                if len(row) == 3:
                    if search_term in row[0].lower() or search_term in row[1].lower() or search_term in row[2].lower():
                        results.append(f"{row[0]} - {row[1]} - {row[2]}")
                    
        if results:
            result_window = Toplevel()
            result_window.title("검색 결과")
            result_window.geometry("400x300")  
            scrollbar = Scrollbar(result_window, orient=VERTICAL)
            result_text = Text(result_window, yscrollcommand=scrollbar.set)
            scrollbar.config(command=result_text.yview)
            scrollbar.pack(side=RIGHT, fill=Y)
            result_text.pack(expand=True, fill='both')
            
            result_text.insert(END, "\n".join(results))
        else:
            messagebox.showinfo("검색 결과", "검색 결과가 없습니다.")
    else:
        messagebox.showwarning("파일 없음", "CSV 파일이 없습니다.")
    
    entry_search.delete(0, tk.END)

def delete_from_csv():
    global csv_file
    if csv_file is None:
        messagebox.showwarning("경로 오류", "먼저 파일 저장 경로를 설정하세요.")
        return
    
    search_term = entry_search.get().lower()
    if not search_term:
        messagebox.showwarning("입력 오류", "삭제할 상품명 또는 번호를 입력하세요.")
        return
    
    if os.path.exists(csv_file):
        updated_rows = []
        found = False
        
        with open(csv_file, 'r') as file:
            reader = csv.reader(file)
            for row in reader:
                if len(row) == 3:
                    if search_term not in row[0].lower() and search_term not in row[1].lower():
                        updated_rows.append(row)
                    else:
                        found = True
        
        if found:
            with open(csv_file, 'w', newline='') as file:
                writer = csv.writer(file)
                writer.writerows(updated_rows)
            messagebox.showinfo("삭제 완료", f"'{search_term}'에 해당하는 항목이 삭제되었습니다.")
        else:
            messagebox.showinfo("삭제 실패", f"'{search_term}'에 해당하는 항목을 찾을 수 없습니다.")
    else:
        messagebox.showwarning("파일 없음", "CSV 파일이 없습니다.")
    
    entry_search.delete(0, tk.END)

def load_csv():
    global csv_file
    if csv_file is None:
        messagebox.showwarning("경로 오류", "먼저 파일 저장 경로를 설정하세요.")
        return
    
    if os.path.exists(csv_file):
        with open(csv_file, 'r') as file:
            reader = csv.reader(file)
            existing_data = list(reader)
            if existing_data:
                messagebox.showinfo("파일 로드 완료", f"{len(existing_data)}개의 항목이 로드되었습니다.")
            else:
                messagebox.showinfo("파일 로드 완료", "파일에 데이터가 없습니다.")
    else:
        messagebox.showwarning("파일 없음", "CSV 파일이 없습니다.")

def clear_fields():
    entry_product.delete(0, tk.END)
    entry_number.delete(0, tk.END)
    entry_price.delete(0, tk.END)
    entry_search.delete(0, tk.END)

root = tk.Tk()
root.title("재고 관리 프로그램")
root.geometry('900x600')

frame = tk.Frame(root)
frame.place(relx=0.5, rely=0.5, anchor='center')

btn_set_path = tk.Button(frame, text="파일 경로 설정", command=set_save_path, width=20, height=2)
btn_set_path.grid(row=0, column=0, padx=10, pady=10)

btn_load = tk.Button(frame, text="파일 로드", command=load_csv, width=20, height=2)
btn_load.grid(row=0, column=1, padx=10, pady=10)

label_product = tk.Label(frame, text="상품명:", font=("Arial", 12))
label_product.grid(row=1, column=0, padx=10, pady=10, sticky='e')

entry_product = tk.Entry(frame, font=("Arial", 12), width=30)
entry_product.grid(row=1, column=1, padx=10, pady=10)

label_number = tk.Label(frame, text="번호:", font=("Arial", 12))
label_number.grid(row=2, column=0, padx=10, pady=10, sticky='e')

entry_number = tk.Entry(frame, font=("Arial", 12), width=30)
entry_number.grid(row=2, column=1, padx=10, pady=10)

label_price = tk.Label(frame, text="가격:", font=("Arial", 12))
label_price.grid(row=3, column=0, padx=10, pady=10, sticky='e')

entry_price = tk.Entry(frame, font=("Arial", 12), width=30)
entry_price.grid(row=3, column=1, padx=10, pady=10)

btn_save = tk.Button(frame, text="저장", command=save_to_csv, width=20, height=2)
btn_save.grid(row=4, column=1, padx=10, pady=10)

label_search = tk.Label(frame, text="검색 또는 삭제 (상품, 번호):", font=("Arial", 12))
label_search.grid(row=5, column=0, padx=10, pady=10, sticky='e')

entry_search = tk.Entry(frame, font=("Arial", 12), width=30)
entry_search.grid(row=5, column=1, padx=10, pady=10)

btn_search = tk.Button(frame, text="검색", command=search_in_csv, width=20, height=2)
btn_search.grid(row=6, column=1, padx=10, pady=10)

btn_delete = tk.Button(frame, text="삭제", command=delete_from_csv, width=20, height=2)
btn_delete.grid(row=7, column=1, padx=10, pady=10)

btn_clear = tk.Button(frame, text="내용 삭제", command=clear_fields, width=20, height=2)
btn_clear.grid(row=8, column=1, padx=10, pady=10)

root.bind('<Return>', save_to_csv)

root.mainloop()